# Determine job-finding rates and stocks
The job-finding rates are functions of the surplus, $\mathcal J(\text{surplus})=mB_2\times\text{surplus}^{1/\alpha}$.

In [ ]:
function JFR(surplus::Float64, C::Union{Country1,Country2,Country1A,Country2A})
    C.m * C.B2 * surplus^(1/C.α)
    #C.m * C.B2 * max(surplus,0)^(1/C.α)
end

function JFR(surplus::Vector{Float64}, C::Union{Country1,Country2,Country1A,Country2A})
    [C.m * C.B2 * S^(1/C.α) for S in surplus]
    #[C.m * C.B2 * max(S,0)^(1/C.α) for S in surplus]
end

# 1) Stocks in a world with unconditional policy
Here, we have the counterfactual dynamics in presence of the policy when everyone is eligible irrespective of their age. 
As with the surpluses, we consider three stocks: 
1. the number of unemployed workers who receive assistance benefits ($b_0$ below $A$ and $b_1$ after),
2. the number of unemployed workers who receive UI benefits for a short period,
3. the number of workers who receive UI benefits for a long period.

In this world, no one is in state 2 (unless there is an initial condition).
The dynamics are governed by:
$$
\begin{align*}
    & \dot{u_1} = q(1-\chi)(1-u_1-u_3)+ \lambda_1 u_3 -\mathcal J(\Sigma_1) u_1,\\
    & u_2 = 0,\\
    & \dot{u_3} = q\chi(1-u_1-u_3) -(\lambda_1+\mathcal J(\Sigma_3)) u_3.
\end{align*}
$$

For the first policy, the initial conditions at age $A_{min}$ are given in the Country object, $(u_1,u_3)=uinit$. 

For the second policy, the dynamics start at the steady state (when $\chi=1$):
$$
\begin{align*}
    & u_1 = \frac{q \lambda_1}{q \lambda_1+\mathcal J(\Sigma_1).(q +\lambda_1+\mathcal J(\Sigma_3))},\\
    & u_3 = \frac{\mathcal J(\Sigma_1)}{\lambda_1} u_1,
\end{align*}
$$
or
\begin{align*}
    & u_1 = \frac{q \lambda_1+q(1-\chi)\mathcal J(\Sigma_3)}
    {\mathcal J(\Sigma_1).(q+\mathcal J(\Sigma_3))+\lambda_1.(q+\mathcal J(\Sigma_1))+q(1-\chi).(\mathcal J(\Sigma_3)-\mathcal J(\Sigma_1))},\\
    & u_3 = \frac{q\chi\mathcal J(\Sigma_1)}
    {\mathcal J(\Sigma_1).(q+\mathcal J(\Sigma_3))+\lambda_1.(q+\mathcal J(\Sigma_1))+q(1-\chi).(\mathcal J(\Sigma_3)-\mathcal J(\Sigma_1))}.
\end{align*}
or approximated at age $A_{min}$ in the life-cycle model  (see code). 

In [ ]:
function ODE_u_counterwith!(du::Vector, u::Vector, V::Surplus, C::Country1)
    du[1] = C.λ * u[2] - JFR(V.Σ[1],C) * u[1]
    du[2] =  C.q * (1-u[1]-u[2]) - ( JFR(V.Σ[2],C) + C.λ) * u[2] #/!\ u[2] is u_3, note that Σ_2=Σ_3
end

function find_u_counterwith(V::Surplus, C::Country1)
    tspan = (C.Amin,C.Amax)
    prob = ODEProblem((du,u,p,t)->ODE_u_counterwith!(du,u,V,C), C.uinit,tspan)
    sol = solve(prob) 
    return sol
end

function find_u_counterwith(V::Surplus, C::Country2)
     ## then start with steady-state condition at Amin of ODE_u_after
    u1 = C.q * C.λ1 / (C.q * C.λ1 + JFR(V.Σ[1],C)*(C.q + JFR(V.Σ[3],C)+C.λ1))
    u3 = u1 *  JFR(V.Σ[1],C) / C.λ1
    return [u1, u3]
end

In [ ]:
## life-cycle policy 1
function ODE_u_counterwith!(du::Vector, u::Vector, a::Float64, V::Surplus, C::Country1A)
    q = qaux(a, C)
    Σ = V.Σ(a)
    du[1] = C.λ * u[2] - JFR(Σ[1],C) * u[1]
    du[2] =  q * (1-u[1]-u[2]) - ( JFR(Σ[2],C) + C.λ) * u[2] #/!\ u[2] is u_3
end

function find_u_counterwith(V::Surplus, C::Country1A)
    tspan = (C.Amin,C.Amax)
    prob = ODEProblem((du,u,p,a)->ODE_u_counterwith!(du,u,a,V,C), C.uinit, tspan)
    sol = solve(prob) 
    return sol
end

## life-cycle policy 2
function ODE_u_counterwith!(du::Vector, u::Vector, a::Float64, V::Surplus, C::Country2A)
    q = qaux(a, C)
    Σ = V.Σ(a)
    du[1] = q * (1-C.χ) * (1-u[1]-u[2]) + C.λ1 * u[2] - JFR(Σ[1],C) * u[1]
    du[2] =  q * C.χ * (1-u[1]-u[2]) - ( JFR(Σ[3],C) + C.λ1) * u[2] #/!\ u[2] is u_3
end

function init_counterwith(V::Surplus, C::Country2A)
    q = qaux(C.Amin, C)
    Σ = V.Σ(C.Amin)
  #=  u1 = q * C.λmin / (q * C.λmin + JFR(Σ[1],C)*(q + JFR(Σ[3],C)+C.λmin))
    u3 = u1 *  JFR(Σ[1],C) / C.λmin =# #for χ=1
    den = JFR(Σ[1],C) * (q + JFR(Σ[3],C)) + C.λ1 * (q + JFR(Σ[1],C)) + q * (1-C.χ) * (JFR(Σ[3],C)-JFR(Σ[1],C))
    u1 = q * (C.λ1 + (1-C.χ) * JFR(Σ[3],C)) / den
    u3 = q * C.χ * JFR(Σ[1],C) / den
    return [u1, u3]
end

function find_u_counterwith(V::Surplus, C::Country2A)
    tspan = (C.Amin,C.Amax)
    uinit = init_counterwith(V, C)
    prob = ODEProblem((du,u,p,a)->ODE_u_counterwith!(du,u,a,V,C), uinit, tspan)
    sol = solve(prob) 
    return sol
end

# 2) Stocks in a world without the policy
In this world, no one is in state 3 (unless there is an initial condition).
The dynamics are governed by:
$$
\begin{align*}
    & \dot{u_1} = q(1-\chi)(1-u_1-u_2)+\lambda_0 u_3 -\mathcal J(\Psi_1) u_1,\\
    & \dot{u_2} = q\chi(1-u_1-u_2) -(\lambda_0+\mathcal J(\Psi_2)) u_2.
\end{align*}
$$
For the first policy, the initial conditions at age $A_{min}$ are given in the Country object, $(u_1,u_2)=uinit$. 
For the second policy, the dynamics start at the steady state when $\chi=1$
$$
\begin{align*}
    & u_1 = \frac{q \lambda_0}{(q \lambda_{0}+\mathcal J(\Psi_1))(q +\lambda_0+\mathcal J(\Psi_2))},\\
    & u_2 = \frac{\mathcal J(\Psi_1)}{\lambda_0} u_1,
\end{align*}
$$
or approximated at age Amin (see code). 

In [ ]:
##counterfactual dynamics without the policy
function ODE_u_counterwithout!(du::Vector, u::Vector, V::Surplus, C::Country1)
    du[1] = C.λ * u[2] - JFR(V.Ψ[1],C) * u[1]
    du[2] = C.q * (1-u[1]-u[2]) - (JFR(V.Ψ[2],C) + C.λ) * u[2]
end

function find_u_counterwithout(V::Surplus, C::Country1)
    tspan = (C.Amin,C.Amax)
    prob = ODEProblem((du,u,p,t)->ODE_u_counterwithout!(du,u,V,C), C.uinit,tspan)
    sol = solve(prob) 
    return sol
end

function find_u_counterwithout(V::Surplus, C::Country2)
    u1 = C.q * C.λ0 / (C.q * C.λ0 + JFR(V.Ψ[1],C)*(C.q + JFR(V.Ψ[2],C)+C.λ0))
    u2 = u1 * JFR(V.Ψ[1],C) / C.λ0
    return [u1, u2]
end

In [ ]:
## life-cycle policy 1
function ODE_u_counterwithout!(du::Vector, u::Vector, a::Float64, V::Surplus, C::Country1A)
    q = qaux(a, C)
    Ψ = V.Ψ(a)
    du[1] = C.λ * u[2] - JFR(Ψ[1],C) * u[1]
    du[2] = q * (1-u[1]-u[2]) - (JFR(Ψ[2],C) + C.λ) * u[2]
end

function find_u_counterwithout(V::Surplus, C::Country1A)
    tspan = (C.Amin,C.Amax)
    prob = ODEProblem((du,u,p,a)->ODE_u_counterwithout!(du,u,a,V,C), C.uinit, tspan)
    sol = solve(prob) 
    return sol
end

## life-cycle policy 2
function ODE_u_counterwithout!(du::Vector, u::Vector, a::Float64, V::Surplus, C::Country2A)
    q = qaux(a, C)
    Ψ = V.Ψ(a)
    du[1] = q * (1-C.χ) * (1-u[1]-u[2]) + C.λ0 * u[2] - JFR(Ψ[1],C) * u[1]
    du[2] = q * C.χ * (1-u[1]-u[2]) - (JFR(Ψ[2],C) + C.λ0) * u[2]
end

function init_counterwithout(V::Surplus, C::Country2A)
    q = qaux(C.Amin, C)
    Ψ = V.Ψ(C.Amin)
    #=u1 = q * C.λmax / (q * C.λmax + JFR(Ψ[1],C)*(q + JFR(Ψ[2],C)+C.λmax))
    u2 = u1 * JFR(Ψ[1],C) / C.λmax=# #for χ=1
    den = JFR(Ψ[1],C) * (q + JFR(Ψ[2],C)) + C.λ0 * (q + JFR(Ψ[1],C)) + q * (1-C.χ) * (JFR(Ψ[2],C)-JFR(Ψ[1],C))
    u1 = q * (C.λ0 + (1-C.χ) * JFR(Ψ[2],C)) / den
    u2 = q * C.χ * JFR(Ψ[1],C) / den
    return [u1, u2]
end

function find_u_counterwithout(V::Surplus, C::Country2A)
    tspan = (C.Amin,C.Amax)
    uinit = init_counterwithout(V, C)
    prob = ODEProblem((du,u,p,a)->ODE_u_counterwithout!(du,u,a,V,C), uinit, tspan)
    sol = solve(prob) 
    return sol
end

# 3) Stocks with age-conditional policy

Before age $A$, we have the dynamics:
$$
\begin{align*}
    & \dot{u_1} = q(1-\chi)(1-u_1-u_2)+\lambda_0 u_2 -\mathcal J(S_1(a)) u_1,\\
    & \dot{u_2} = q\chi(1-u_1-u_2) -(\lambda_0+\mathcal J(S_2(a)) u_2,\\
    & u_3 = 0.
\end{align*}
$$

For the first policy, the initial conditions at age $A_{min}$ are given in the Country object, $(u_1,u_2)=uinit$, and always $\chi=1$. 
For the second policy, the economy starts at the initial state of the counterfactual dynamics without policy, corresponding to a myopic steady state.

After age $A$, the dynamics are:
$$
\begin{align*}
    & \dot{u_1} = q(1-\chi)(1-u_1-u_2-u_3)+\lambda_0u_2 + \lambda_1u_3 -\mathcal J(\Sigma_1) u_1,\\
    & \dot{u_2} = -(\lambda_0+\mathcal J(\Sigma_2)) u_2,\\
    & \dot{u_3} = q\chi(1-u_1-u_2-u_3) -(\lambda_1+\mathcal J(\Sigma_3)) u_3.
\end{align*}
$$
with initial conditions at age $A$ given by the solution of the previous system.

For the first policy, we ignore state 3 and $\chi=1$ so the system is simply
$$
\begin{align*}
    & \dot{u_1} = \lambda u_2  -\mathcal J(\Sigma_1) u_1,\\
    & \dot{u_2} = q(1-u_1-u_2) -(\lambda+\mathcal J(\Sigma_2)) u_2.
\end{align*}
$$

In [ ]:
## first policy
function ODE_u_realbefore(du::Vector, u::Vector, a::Float64, V::Surplus, C::Country1)
    du[1] = C.λ * u[2] - JFR(V.S(a)[1],C) * u[1]
    du[2] = C.q * (1-u[1]-u[2]) - (JFR(V.S(a)[2],C) + C.λ) * u[2]
end

function find_u_realbefore(V::Surplus, C::Country1)
    tspan = (C.Amin,C.A)
    prob = ODEProblem((du,u,p,a)->ODE_u_realbefore(du,u,a,V,C), C.uinit,tspan)
    sol_before = solve(prob)
    return sol_before
end

function ODE_u_realafter(du::Vector, u::Vector, V::Surplus, C::Country1)
    du[1] = C.λ * u[2] - JFR(V.Σ[1],C) * u[1]
    du[2] =  C.q * (1-u[1]-u[2]) - ( JFR(V.Σ[2],C) + C.λ) * u[2]  ## note Σ_2=Σ_3
end

function find_u_realafter(uinitA::Vector, V::Surplus, C::Country1)
    tspan = (C.A,C.Amax)
    prob = ODEProblem((du,u,p,t)->ODE_u_realafter(du,u,V,C),uinitA,tspan)
    sol_after = solve(prob)
    return sol_after
end

function find_u_real(V::Surplus, C::Country1)
    ureal_bef = find_u_realbefore(V, C)
    uinitA = ureal_bef(C.A)
    ureal_aft = find_u_realafter(uinitA, V, C)
    function ureal(a)
        if a < C.A
            ureal_bef(a)
        else
            ureal_aft(a)
        end
    end
    return ureal
end


## second policy
function ODE_u_realbefore(du::Vector, u::Vector, a::Float64, V::Surplus, C::Country2)
    du[1] = C.λ0 * u[2] - JFR(V.S(a)[1],C) * u[1]
    du[2] = C.q * (1-u[1]-u[2]) - (JFR(V.S(a)[2],C) + C.λ0) * u[2]
end

function find_u_realbefore(V::Surplus, C::Country2)
    u1 = C.q * C.λ0 / (C.q * C.λ0 + JFR(V.Ψ[1],C)*(C.q + JFR(V.Ψ[2],C)+C.λ0))
    u2 = u1 * JFR(V.Ψ[1],C) / C.λ0
    uinit = [u1, u2]
    tspan = (C.Amin,C.A)
    prob = ODEProblem((du,u,p,a)->ODE_u_realbefore(du,u,a,V,C),uinit,tspan)
    sol_before = solve(prob) 
    return a -> vcat(sol_before(a), 0.)
end

function ODE_u_realafter(du::Vector, u::Vector, a::Float64, V::Surplus, C::Country2)
    du[1] = C.λ0 * u[2] + C.λ1 * u[3] - JFR(V.Σ[1],C) * u[1]
    du[2] = - (JFR(V.Σ[2],C) + C.λ0) * u[2]
    du[3] =  C.q * (1-u[1]-u[2]-u[3]) - ( JFR(V.Σ[3],C) + C.λ1) * u[3]
end

function find_u_realafter(uinitA::Vector, V::Surplus, C::Country2)
    tspan = (C.A,C.Amax)
    prob = ODEProblem((du,u,p,t)->ODE_u_realafter(du,u,t,V,C),uinitA,tspan)
    sol_after = solve(prob) 
    return a->sol_after(a)
end

function find_u_real(V::Surplus, C::Country2)
    ureal_bef = find_u_realbefore(V, C)
    uinitA = ureal_bef(C.A)
    ureal_aft = find_u_realafter(uinitA, V, C)
    function ureal(a)
        if a<C.A
            ureal_bef(a)
        else
            ureal_aft(a)
        end
    end
    return ureal 
end

In [ ]:
## life-cycle policy 1
function ODE_u_realbefore(du::Vector, u::Vector, a::Float64, V::Surplus, C::Country1A)
    q = qaux(a, C)
    S = V.S(a)
    du[1] = C.λ * u[2] - JFR(S[1],C) * u[1]
    du[2] = q * (1-u[1]-u[2]) - (JFR(S[2],C) + C.λ) * u[2]
end

function find_u_realbefore(V::Surplus, C::Country1A)
    tspan = (C.Amin,C.A)
    prob = ODEProblem((du,u,p,a)->ODE_u_realbefore(du,u,a,V,C), C.uinit,tspan)
    sol_before = solve(prob)
    return sol_before
end

function ODE_u_realafter(du::Vector, u::Vector, a::Float64, V::Surplus, C::Country1A)
    q = qaux(a, C)
    Σ = V.Σ(a)
    du[1] = C.λ * u[2] - JFR(Σ[1],C) * u[1]
    du[2] =  q * (1-u[1]-u[2]) - ( JFR(Σ[2],C) + C.λ) * u[2]  ## note Σ_2=Σ_3
end

function find_u_realafter(uinitA::Vector, V::Surplus, C::Country1A)
    tspan = (C.A,C.Amax)
    prob = ODEProblem((du,u,p,a)->ODE_u_realafter(du,u,a,V,C),uinitA,tspan)
    sol_after = solve(prob) 
    return sol_after
end

function find_u_real(V::Surplus, C::Country1A)
    ureal_bef = find_u_realbefore(V, C)
    uinitA = ureal_bef(C.A)
    ureal_aft = find_u_realafter(uinitA, V, C)
    function ureal(a)
        if a < C.A
            ureal_bef(a)
        else
            ureal_aft(a)
        end
    end
    return ureal
end


## life-cycle policy 2
function ODE_u_realbefore(du::Vector, u::Vector, a::Float64, V::Surplus, C::Country2A)
    q = qaux(a, C)
    S = V.S(a)
    du[1] = q * (1-C.χ) * (1-u[1]-u[2]) + C.λ0 * u[2] - JFR(S[1],C) * u[1]
    du[2] = q * C.χ * (1-u[1]-u[2]) - (JFR(S[2],C) + C.λ0) * u[2]
end

function find_u_realbefore(uinit::Vector, V::Surplus, C::Country2A)
    tspan = (C.Amin,C.A)
    prob = ODEProblem((du,u,p,a)->ODE_u_realbefore(du,u,a,V,C),uinit,tspan)
    sol_before = solve(prob) 
    return a -> vcat(sol_before(a), 0.)
end

function ODE_u_realafter(du::Vector, u::Vector, a::Float64, V::Surplus, C::Country2A)
    q = qaux(a, C)
    Σ = V.Σ(a)
    du[1] = q * (1-C.χ) * (1-u[1]-u[2]-u[3]) + C.λ0 * u[2] + C.λ1 * u[3] - JFR(Σ[1],C) * u[1]
    du[2] = - (JFR(Σ[2],C) + C.λ0) * u[2]
    du[3] =  q * C.χ * (1-u[1]-u[2]-u[3]) - ( JFR(Σ[3],C) + C.λ1) * u[3]
end

function find_u_realafter(uinitA::Vector, V::Surplus, C::Country2A)
    tspan = (C.A,C.Amax)
    prob = ODEProblem((du,u,p,a)->ODE_u_realafter(du,u,a,V,C),uinitA,tspan)
    sol_after = solve(prob) 
    return a->sol_after(a)
end

function find_u_real(V::Surplus, C::Country2A)
    uinit = init_counterwithout(V, C)
    ureal_bef = find_u_realbefore(uinit, V, C)
    uinitA = ureal_bef(C.A)
    ureal_aft = find_u_realafter(uinitA, V, C)
    function ureal(a)
        if a<C.A
            ureal_bef(a)
        else
            ureal_aft(a)
        end
    end
    return ureal 
end

# 4) Wrap-up

In [ ]:
function find_Unemp(V::Surplus, C::Union{Country1, Country2, Country1A, Country2A}; 
        find_with=true, find_real=true, find_without=true)
    with = find_with ? find_u_counterwith(V, C) : zeros(2)
    without = find_without ? find_u_counterwithout(V, C) : zeros(2)
    real = find_real ? find_u_real(V, C) : zeros(3)
    return Unemp(with=with, real=real, without=without)
end